In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame,read_csv
from sklearn.utils import shuffle

In [2]:
df = read_csv('heart_failure.csv')
df.head(5)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [3]:
df['anaemia'] = np.where(df['anaemia'] == 1 ,True,False)
df['diabetes'] = np.where(df['diabetes'] == 1, True, False)
df['high_blood_pressure'] = np.where(df['high_blood_pressure'] == 1, True, False)
df['smoking'] = np.where(df['smoking'] == 1, True, False)
df['sex'] = np.where(df['sex'] == 1, 'Male','Female')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    bool   
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    bool   
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    bool   
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    object 
 10  smoking                   299 non-null    bool   
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: bool(4), float64(3), int64(5), object(1)
memory usage: 22.3+ K

In [5]:
X = df.drop('DEATH_EVENT', axis = 1)
y = df['DEATH_EVENT']
print(X.shape)
X.head(2)

(299, 12)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,False,582,False,20,True,265000.00,1.9,130,Male,False,4
1,55.0,False,7861,False,38,False,263358.03,1.1,136,Male,False,6


In [17]:
import imblearn
from imblearn.over_sampling import SMOTENC
smote = SMOTENC(random_state=1,categorical_features=[0,1,3,5,9,10])
x_bal, y_bal = smote.fit_sample(X,y)
x_bal = DataFrame(x_bal, columns = X.columns)
print(Series(y_bal).value_counts())

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


1    203
0    203
dtype: int64


In [18]:
x = pd.get_dummies(x_bal,columns = ['sex'], drop_first = True)
print(x.shape)
x.head()

(406, 12)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,smoking,time,sex_Male
0,75,False,582,False,20,True,265000,1.9,130,False,4,1
1,55,False,7861,False,38,False,263358,1.1,136,False,6,1
2,65,False,146,False,20,False,162000,1.3,129,True,7,1
3,50,True,111,False,20,False,210000,1.9,137,False,7,1
4,65,True,160,True,20,False,327000,2.7,116,False,8,0


In [19]:
col = ['age','creatinine_phosphokinase','ejection_fraction',
       'platelets','serum_creatinine','serum_sodium','time',
       'anaemia','diabetes','high_blood_pressure','smoking',
       'sex_Male']

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

col_trans = ColumnTransformer(remainder='passthrough',
                              transformers = [('scaler',MinMaxScaler(),[0,2,4,6,7,8,10])])
trans = col_trans.fit_transform(x)
trans = DataFrame(trans,columns = col)
trans.head()

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time,anaemia,diabetes,high_blood_pressure,smoking,sex_Male
0,0.636364,0.0713192,0.0909091,0.290823,0.157303,0.485714,0,False,False,True,False,1
1,0.272727,1,0.363636,0.288833,0.0674157,0.657143,0.00711744,False,False,False,False,1
2,0.454545,0.0156928,0.0909091,0.16596,0.0898876,0.457143,0.0106762,False,False,False,True,1
3,0.181818,0.0112274,0.0909091,0.224148,0.157303,0.685714,0.0106762,True,False,False,False,1
4,0.454545,0.0174789,0.0909091,0.365984,0.247191,0.0857143,0.0142349,True,True,False,False,0


In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(trans,y_bal, test_size = 0.2, random_state = 0)

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
grid = GridSearchCV(RandomForestClassifier(),
                {'max_depth':range(1,15), 'n_estimators': [50, 100,300],
                'max_features': ['auto', 'sqrt', 'log'], 
                'min_samples_split': [2,3,5,7,9],'min_samples_leaf': [1,2,4,6,8]},
                 cv = 3, n_jobs = 2, scoring = 'recall', verbose = 1)

grid.fit(x_train,y_train)

Fitting 3 folds for each of 3150 candidates, totalling 9450 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    9.7s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   37.3s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  1.4min
[Parallel(n_jobs=2)]: Done 1108 tasks      | elapsed:  2.9min
[Parallel(n_jobs=2)]: Done 2008 tasks      | elapsed:  4.8min
[Parallel(n_jobs=2)]: Done 3108 tasks      | elapsed:  7.8min
[Parallel(n_jobs=2)]: Done 4408 tasks      | elapsed: 11.0min
[Parallel(n_jobs=2)]: Done 5908 tasks      | elapsed: 14.6min
[Parallel(n_jobs=2)]: Done 7608 tasks      | elapsed: 18.7min
[Parallel(n_jobs=2)]: Done 9450 out of 9450 | elapsed: 23.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [24]:
grid.best_params_

{'max_depth': 9,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 7,
 'n_estimators': 50}

In [130]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_features= 'auto',random_state = 3,
                                    min_samples_leaf = 1,
                                    min_samples_split = 2,n_estimators = 100)
classifier.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=3, verbose=0,
                       warm_start=False)

In [133]:
y_pred = classifier.predict(x_test)

In [134]:
from sklearn.metrics import classification_report, recall_score, accuracy_score,precision_score, f1_score, confusion_matrix
recall = recall_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
f1score = f1_score(y_test,y_pred)
print('the Recall for tuned RandomForest is:{}'.format(round(recall,4)))
print(f'the Accuracy for tuned RandomForest is:{round(accuracy,4)}')
print('the Precision for tuned RandomForest is: %s' %(round(precision,4)))
print(f'the F1_score for tuned RandomForest is:{round(f1score,4)}')

the Recall for tuned RandomForest is:0.9787
the Accuracy for tuned RandomForest is:0.9512
the Precision for tuned RandomForest is: 0.9388
the F1_score for tuned RandomForest is:0.9583


In [135]:
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.91      0.94        35
           1       0.94      0.98      0.96        47

    accuracy                           0.95        82
   macro avg       0.95      0.95      0.95        82
weighted avg       0.95      0.95      0.95        82



In [136]:
matrix = confusion_matrix(y_test,y_pred)
print(matrix)

[[32  3]
 [ 1 46]]


In [137]:
import pickle
pickle.dump(classifier, open('model.pkl','wb'))